# 🚀 Basic LLM Fine-Tuning with Hugging Face

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Gaurav14cs17/LLMs_Model/blob/main/Fine-Tuning-LLMs-Guide/notebooks/01_basic_fine_tuning.ipynb)

This notebook demonstrates basic **Supervised Fine-Tuning (SFT)** of a Large Language Model.

### 📋 What You'll Learn
- Load a pre-trained model from Hugging Face
- Prepare instruction-following dataset  
- Fine-tune using the SFTTrainer API
- Run inference with your fine-tuned model

**⚠️ Requirements**: GPU with 16GB+ VRAM (T4 on free Colab works!)


In [ ]:
# Install dependencies
!pip install -q transformers datasets accelerate peft bitsandbytes trl

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset

print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")


In [ ]:
# Configuration
MODEL_NAME = "microsoft/phi-2"
OUTPUT_DIR = "./fine-tuned-model"
MAX_LENGTH = 512

# Load dataset
dataset = load_dataset("tatsu-lab/alpaca", split="train[:1000]")
print(f"Dataset: {len(dataset)} samples")


In [ ]:
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, 
    torch_dtype=torch.float16, 
    device_map="auto"
)
print("Model loaded!")


In [ ]:
# Format dataset for instruction tuning
def format_instruction(sample):
    """Format each sample as instruction-input-output"""
    if sample.get("input", ""):
        text = f"""### Instruction:
{sample['instruction']}

### Input:
{sample['input']}

### Response:
{sample['output']}"""
    else:
        text = f"""### Instruction:
{sample['instruction']}

### Response:
{sample['output']}"""
    return {"text": text}

# Apply formatting
dataset = dataset.map(format_instruction)
print(f"Sample:\n{dataset[0]['text'][:500]}...")


In [ ]:
# Training with SFTTrainer (recommended for instruction tuning)
from trl import SFTTrainer, SFTConfig

training_args = SFTConfig(
    output_dir=OUTPUT_DIR,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    warmup_ratio=0.03,
    logging_steps=10,
    save_steps=100,
    fp16=True,
    max_seq_length=MAX_LENGTH,
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
)

print("🚀 Starting training...")
trainer.train()
print("✅ Training complete!")


In [ ]:
# Save the fine-tuned model
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"💾 Model saved to {OUTPUT_DIR}")


In [ ]:
# 🧪 Test the fine-tuned model
def generate_response(prompt, max_new_tokens=128):
    """Generate response from fine-tuned model"""
    formatted = f"### Instruction:\n{prompt}\n\n### Response:\n"
    inputs = tokenizer(formatted, return_tensors="pt").to(model.device)
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("### Response:")[-1].strip()

# Test it!
test_prompt = "Explain what machine learning is in simple terms."
print(f"📝 Prompt: {test_prompt}")
print(f"🤖 Response: {generate_response(test_prompt)}")


## 🎉 Congratulations!

You've successfully fine-tuned an LLM! Next steps:
- Try [LoRA Fine-Tuning](./02_lora_fine_tuning.ipynb) for more memory-efficient training
- Try [QLoRA](./03_qlora_fine_tuning.ipynb) if you have limited GPU memory
- Explore [DPO Training](./04_dpo_training.ipynb) for preference alignment

📚 Reference: [A Comprehensive Guide to Fine-Tuning LLMs](https://arxiv.org/html/2408.13296v1)
